# **Installing packages and importing libraries**

In [ ]:

!pip install -q pyomo
from pyomo.environ import *
import numpy as np
import pandas as pd

!apt-get install -y -qq glpk-utils

!apt-get install -y -qq coinor-cbc



     |████████████████████████████████| 9.4MB 2.7MB/s 
     |████████████████████████████████| 51kB 4.4MB/s 
     |████████████████████████████████| 256kB 21.2MB/s 
     |████████████████████████████████| 163kB 20.2MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 144676 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-

# **Ex2:  *Problem formulation*

 \begin{array}{l}
Let\ b_{0} \ ,\ b_{1} \ and\ b_{2} \ be\ the\ decision\ variables\ as\ described\ in\ the\ question.\\
This\ problem\ reduces\ to\ solving\ the\ following\ optimization\ problem\\
\\
Min\ \ \ \sum ^{6}_{i=1} |R_{i} |\\
where\ \ \ \ R_{i} =\ P_{i} \ -\widetilde{P_{i}}\\
and\ \ \ \ \ \tilde{P}_{i} \ =b_{0} +b_{1} L_{i} +b_{2} E_{i} \ \ \ \ \ ;\ \ i=\ 1,2,\ .\ .\ .\ ,6.\\
\\
s.t\\
b\geqslant 0
\end{array}


# **Approach 1:**   *Problem, Formulationa and solver* 

 \begin{array}{l}
Let\ \ \ \ \ \ \ \ \ \ e_{i} \ =\ |R_{i} |\\
Then\ the\ objective\ funtion\ becomes\ :\\
\\
Min\ \ \ \ \ \ \sum ^{6}_{i=1} e_{i}\\
and\ the\ constraits\ are\ \\
\\
e_{i} \geqslant 0\\
e_{i} \geqslant \ R_{i} \ \ \ \ or\ \ \ \ e_{i} -R_{i} \ \geqslant 0\ \ \ \ or\ \ \ \ e_{i} -P_{i} +b_{0} +b_{1} L_{i} +b_{2} E_{i} \ \geqslant 0\\
e_{i} \geqslant -R_{i} \ \ \ \ or\ \ \ \ e_{i} +R_{i} \ \geqslant 0\ \ \ or\ \ \ \ \ e_{i} \ +P_{i} -b_{0} -b_{1} L_{i} -b_{2} E_{i} \geqslant 0 \tag{}\\
\\
for\ i=1,2...5,6\\
\end{array}


In [ ]:
data = pd.read_csv('data.csv')
data

model1 = ConcreteModel()
N = len(data.columns)-1
M = len(data)
b_index = np.arange(N)
e_index = np.arange(M)

model1.b = Var(b_index,domain = Reals)
model1.e = Var(e_index,domain= NonNegativeReals)

model1.obj = Objective(expr = sum(model1.e[i] for i in e_index),sense =minimize)
model1.cons = ConstraintList()

for i in e_index:
  model1.cons.add(model1.e[i] - data['Pi'][i] + summation(data.iloc[i,1:],model1.b) >= 0)

for i in e_index:
  model1.cons.add(model1.e[i] + data['Pi'][i] - summation(data.iloc[i,1:],model1.b) >= 0)

model1.b[0].setlb(0)
model1.pprint()

cbc = SolverFactory('cbc')
result1 = cbc.solve(model1)



print('Status :',result1.solver.status)
print('Termination Condition :',result1.solver.termination_condition)
print('The optimal vlaue of the objective function is :',model1.obj())
print('\n')
print('The value of the variables are : ')
print('\n')
for i in b_index:
  print('[b',i,'] :', model1.b[i].value)

for i in e_index:
  print('[e',i,'] :', model1.e[i].value)



3 Set Declarations
    b_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}
    cons_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
    e_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

2 Var Declarations
    b : Size=3, Index=b_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True :  Reals
          1 :  None :  None :  None : False :  True :  Reals
          2 :  None :  None :  None : False :  True :  Reals
    e : Size=6, Index=e_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  

# **Approach 2:**   *Problem, Formulationa and solver* 

 \begin{array}{l}
Lets\ write\ the\ above\ problem\ in\ the\ following\ way\ :\\
\\
|R_{i} |\ =\ a_{i} +c_{i}\\
R_{i} \ =\ a_{i} -c_{i}\\
Then\ the\ objective\ funtion\ becomes\ :\\
\\
Min\ \ \ \ \ \ \sum ^{6}_{i=1}( a_{i} +c_{i})\\
and\ the\ constraits\ are\ \\
a_{i} \ ,\ c_{i} \ \geqslant 0\\
P_{i} \ -b_{0} -b_{1} L_{i} -b_{2} E_{i} -a_{i} +c_{i} =0
\end{array}

In [ ]:
data2 = pd.read_csv('data2.csv')
data2

model2 = ConcreteModel()
N2 = len(data2.columns)-1
M2 = len(data2)

col_index = np.arange(N2)
b_index = np.arange(3)
row_index = np.arange(M2)

model2.b = Var(b_index,domain = Reals)
model2.a = Var(row_index, domain=NonNegativeReals)
model2.c = Var(row_index, domain=NonNegativeReals)

model2.obj = Objective(expr = sum(model2.a[i] + model2.c[i] for i in row_index),sense =minimize)
model2.cons = ConstraintList()

for i in row_index:
  model2.cons.add(data2['Pi'][i] + summation(data2.iloc[i,1:4],model2.b) + data2.iloc[i,4] * model2.a[i] + data2.iloc[i,5] * model2.c[i] == 0)



model2.b[0].setlb(0)
model2.pprint()

cbc = SolverFactory('cbc')
result2 = cbc.solve(model2)



print('Status :',result2.solver.status)
print('Termination Condition :',result2.solver.termination_condition)
print('The optimal vlaue of the objective function is :',model2.obj())
print('\n')
print('The value of the variables are : ')
print('\n')
for i in b_index:
  print('[b',i,'] :', model2.b[i].value)

for i in row_index:
  print('[a',i,'] :', model2.a[i].value)
  print('[c',i,'] :', model2.c[i].value)



4 Set Declarations
    a_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}
    b_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}
    c_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}
    cons_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {1, 2, 3, 4, 5, 6}

3 Var Declarations
    a : Size=6, Index=a_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  

# **Comment**

The optimal value in both the problem is same as we expected. 

In the first approach the errors are given by sum of ei and in the second the eroors are given by total of ai+bi